In [4]:
from glfinancial import *
import scipy.optimize
import matplotlib.pyplot as plt
import numpy as np

pretax_income = np.arange(75 * 1000, 350 * 1000, 10 * 1000)
posttax_income = posttax(pretax_income)
posttax_income_as_percent = posttax_income/pretax_income
polyfit_order = 1
approx_percent_polynom = np.poly1d(np.polyfit(pretax_income, posttax_income/pretax_income, polyfit_order))
approx_percent = approx_percent_polynom(pretax_income)
approximation_error = [abs(x-y) for x, y  in zip(approx_percent, posttax_income_as_percent)]
avg_approx_error = np.average(approximation_error)
approximation_accuracy = [1-x for x in approximation_error]

print('Pretax Income ranging from {} to {}'.format(pretax_income[0], pretax_income[-1]))
print('Avg approx error is {:.2}'.format(avg_approx_error))
print('Polynomial terms are {}'.format(np.asarray(approx_percent_polynom)))

fig, ax = plt.subplots()
plt.plot(pretax_income, posttax_income_as_percent, 'og', ms=5, label='2019 Federal Tax Rates (Single Filer)')
plt.plot(pretax_income, approx_percent, '-r', ms=5, label='Polynomial Fit ({} order)'.format(polyfit_order))
plt.plot(pretax_income, approximation_accuracy, 'xb', ms=5, label='Approximation Accuracy')
plt.axis([50000,pretax_income[-1],0.65,1])
ax.legend()
ax.set_xlabel('PreTax Income ($)')
ax.set_ylabel('% of PreTax Income')
plt.grid()
plt.show()

fm = FinancialModel(year=2019)

sim_results = [] # We'll append all sim results to this list for one large fm plot at the end
ho_single = CashflowModel('Home Ownership (Single/NonCA)') #Home Ownership Model (as compared to renting, etc)
cm = ho_single
cm.add_yearly(100000, 'Salary', year_end=20, taxable=True)
cm.add_monthly(-2000, 'Pri. Res.', monthly_nw=2000, year_end=15)
cm.add_monthly(-1000, 'Sec. Res.', monthly_nw=1000, year_start=4, year_end=19)
cm.add_monthly(-400, 'Auto Costs', yearly_apr=1.01) # Two vehicles - car and truck, one as a 'beater'
cm.add_yearly(-19000, '401k Savings', yearly_nw=19000, year_end=25, taxable=True)
cm.add_yearly(-5000, 'IRA Savings', yearly_nw=5000, year_end=25)
cm.add_monthly(-3000, 'Monthly Spend')
cm.add_yearly(-12000, 'Yearly Spend')


renting_single = CashflowModel('Renting Model (Single/CA)', state_tax=ca_tax_rates)
cm = renting_single
cm.add_yearly(120000, 'Salary', year_end=20, taxable=True)
cm.add_monthly(-2500, 'Primary Residence', monthly_nw=0)
cm.add_monthly(-400, 'Auto Costs', yearly_apr=1.01)
cm.add_yearly(-19000, '401k Savings', yearly_nw=19000, year_end=20, taxable=True)
cm.add_yearly(-5000, 'IRA Savings', yearly_nw=5000, year_end=20)
cm.add_monthly(-3300, 'Monthly Spend') # CA is expensive
cm.add_yearly(-13500, 'Yearly Spend') # CA is expensive

sims_to_run = [ho_single, renting_single]
sim_results = []

for sim in sims_to_run:
    sim_results.append(fm.sim(sim, nyears=25, initial_nw=100000, nw_apr_avg=1.07, nw_apr_stdev=0.05))
# The default suffix is just standard deviation. The :0.80 suffix gives me the 10th and 90th percentile bars on plots
fm.plot(sim_results, suffix=':0.80')



Pretax Income ranging from 75000 to 345000
Avg approx error is 0.0038
Polynomial terms are [-4.94658938e-07  8.94934191e-01]


<Figure size 640x480 with 1 Axes>

Starting sim for Home Ownership (Single/NonCA)+100000 @ 1.07+/-0.05 for 25 yrs ... Finished in 2.6 seconds
Starting sim for Renting Model (Single/CA)+100000 @ 1.07+/-0.05 for 25 yrs ... Finished in 2.3 seconds


NameError: name 'plt' is not defined